![](post.png)

<!-- metadata: title -->
# Shazam Playlist to Youtube Playlist

<!-- metadata: subtitle -->
> ### Play Shazam playlist from Youtube

<!-- metadata: keywords, type=array -->
**Keywords:**
  - shazam
  - playlist
  - youtube

<!-- metadata: categories, type=array -->
**Categories:**
  - ideas
  - shazam
  - youtube

<!-- metadata: -->
**Description:**
<!-- metadata: description -->
This is a simple idea of a utility app to play songs discovered by [Shazam](https://www.shazam.com/myshazam). The songs are played from youtube.

## Imports

In [1]:
import pandas as pd
from pytube import Search, YouTube
import gradio as gr
from gradio.utils import NamedString
import json

## Preview Data

In [2]:
shazamlibrary_df = pd.read_csv('shazamlibrary.csv', header=1)
shazamlibrary_df

,Index,TagTime,Title,Artist,URL,TrackKey
0,1,2024-06-13,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold,https://www.shazam.com/track/581828191/until-i...,581828191
1,2,2024-06-13,Capitals of Europe: Nordic Region,Katrina Holland,https://www.shazam.com/track/651866832/capital...,651866832
2,3,2024-06-13,Capitals of Europe: Nordic Region,Katrina Holland,https://www.shazam.com/track/651866832/capital...,651866832
3,4,2024-06-11,Ven Ven,Lotus Beatz,https://www.shazam.com/track/696779462/ven-ven,696779462
4,5,2024-05-30,Ava,Famy,https://www.shazam.com/track/127047942/ava,127047942
...,...,...,...,...,...,...
222,223,2018-03-10,Lose Yourself,Felax,https://www.shazam.com/track/340620006/lose-yo...,340620006
223,224,2018-03-08,This Is What It Feels Like (feat. Trevor Guthrie),Armin van Buuren,https://www.shazam.com/track/89020928/this-is-...,83795151
224,225,2018-03-04,Roulette,Katy Perry,https://www.shazam.com/track/358615884/roulette,358615884
225,226,2018-03-04,Roulette,Katy Perry,https://www.shazam.com/track/358615884/roulette,358615884


## Remove Duplicates and Select `Title` and `Artist`.

In [3]:
shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
shazamlibrary_df

,Title,Artist
0,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold
1,Capitals of Europe: Nordic Region,Katrina Holland
3,Ven Ven,Lotus Beatz
4,Ava,Famy
5,Hero,Bryan Todd Feat. Ashley Argota
...,...,...
221,恋人心,Wei Xin Yu
222,Lose Yourself,Felax
223,This Is What It Feels Like (feat. Trevor Guthrie),Armin van Buuren
224,Roulette,Katy Perry


In [8]:
def get_youtube_song(title: str, artist: str) -> YouTube | None:
    search_result = Search(f'{title} by {artist}')
    return search_result.results[0] if search_result.results else None

In [83]:
search_title, search_artist = shazamlibrary_df.loc[10].values
youtube: YouTube = get_youtube_song(search_title, search_artist)
youtube.watch_url

Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Stay With Me by CHANYEOL & Punch
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Stay With Me by CHANYEOL & Punch
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.


'https://youtube.com/watch?v=A8syhfGRx4A'

## Code to run the player

In [103]:
#| output: false

youtube_layer_id = 'youtube-player'
def embed_html(playlist: pd.DataFrame):
    video_ids = [get_youtube_song(*playlist.iloc[i].values).video_id for i in range(len(playlist.head(4)))]
    video_ids_str: str = json.dumps(video_ids)
    player_html = f"<div id='{youtube_layer_id}' data-video_ids='{video_ids_str}'></div>"
    return gr.HTML(player_html, label="Youtube Player", visible=True)

def upload_file(file: NamedString):
    shazamlibrary_df = pd.read_csv(file, header=1)
    shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
    playlist = gr.DataFrame(
        shazamlibrary_df, 
        label="Playlist", 
        visible=True,
        column_widths = ['60%', '40%'],
        col_count=(2, 'fixed'),
        row_count=(len(shazamlibrary_df), 'fixed'))
    player = embed_html(shazamlibrary_df)
    return (playlist, player)
# NOT PLAYING FIRST SONG IN PLAYLIST
body_head = f'''
    <script async src="https://www.youtube.com/iframe_api"></script>
    <style>
        iframe, #{youtube_layer_id} {{
            height:100%; 
            width:100%; 
            min-height: 50vh;
        }}
    </style>
    <script>
        function initPlayer (youtube_player_el, nextVideo) {{
            const youtube_player = window.youtube_player = new YT.Player(youtube_player_el, {{
                height: '100%',
                width: '100%',
                playerVars: {{ autoplay: 1 }},
                events: {{
                    'onReady': function (event) {{
                        onContinue();
                    }},
                    'onStateChange': function (event) {{
                        if (event.data === YT.PlayerState.ENDED) {{
                            onContinue();
                        }}
                    }},
                    'onError': onContinue,
                    'onAutoplayBlocked': function (event) {{
                        (event.target || youtube_player).playVideo();
                    }}
                }}
            }});

            function onContinue() {{
                console.log('onContinue')
                nextVideo((value) => {{
                    youtube_player.loadVideoById(value);
                    youtube_player.playVideo();
                }})
            }}
        }};
        
        const setIntervalId = setInterval(() => {{
            const youtube_player_el = document.getElementById('{youtube_layer_id}');
            if (youtube_player_el) {{
                clearInterval(setIntervalId);
                const video_ids = JSON.parse(youtube_player_el.dataset.video_ids);
                function* generateVideoIds() {{
                    for (const video_id of video_ids) {{
                        yield video_id
                    }}
                }}
                let videoIdGenerator = generateVideoIds();
                function nextVideo(callback) {{
                    let video_id = videoIdGenerator.next().value;
                    if (!video_id) {{
                        videoIdGenerator = generateVideoIds();
                        video_id = videoIdGenerator.next().value;
                    }}
                    callback(video_id);
                }}

                initPlayer(youtube_player_el, nextVideo);
            }}
        }}, 500);
    </script>
'''

with gr.Blocks(head=body_head) as app:
    gr.Markdown("<h1><center>Play your Shazam Playlist from Youtube</center></h1>")
    gr.Markdown("<center>Download the CSV of your playlist from <https://www.shazam.com/myshazam>. </center>")
    gr.Markdown("<center>Upload your Shazam Playlist CSV file.</center>")
    gr.HTML('<hr/>')
    csv = gr.UploadButton(
        label='Upload Shazam Playlist CSV',
        file_count="single",
        file_types = ['.csv'])
    with gr.Row():
        with gr.Column(2):
            player = gr.HTML(label="Youtube Player", visible=False)
        with gr.Column(1):
            playlist = gr.DataFrame(label="Playlist", visible=False)
    csv.upload(
        upload_file, 
        csv, 
        [playlist, player])

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Until I Found You (Em Beihold Version) by Stephen Sanchez & Em Beihold
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Until I Found You (Em Beihold Version) by Stephen Sanchez & Em Beihold
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Ven Ven by Lotus Beatz
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Ven Ven by Lotus Beatz
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reel

Keyboard interruption in main thread... closing server.


## Test the App

> <https://toknow-ai-shazam-to-youtube-playlist.hf.space>

In [96]:
%%HTML

<iframe
	src="https://toknow-ai-shazam-to-youtube-playlist.hf.space"
	frameborder="0"
	width="850"
	height="450"
></iframe>